# Project 1: Building a Second-Price Auction (Modeling and Strategy)
## Implementing the multi-armed bandit algorithm

-----

## Auction Structure and Rules

A second-price auction is conducted for a randomly chosen User with
equal probability in each round (a user may be chosen during more than 
one round). 
1. User navigates to a website with an ad space
2. Bidders place bids and the winner shows the User their ad.
    - Bidders are all given user_id and get to make a bid (any
    non negative amount of $). Bidders don't know how much any
    other Bidder has bid
    - Winner is Bidder with highest bid. If there is a tie, one of
    the highest bidders is selected at random, each with equal
    probability
    - Winning price is second-highest bid meaning the max bid after
    the winner's bid is removed from the set of all bids
    - note if more than one bidder submitted the max bid (tie) then 
    the second price (winning price) will be the max bid (ex two people bid 2
    and no else bids higher than 2, then 2 is the winning price)
3. User may/may not click the ad and winning Bidder observes this.
    - User is shown ad, clicks or doesnt click according to their secret probabiliy
    - the winning bidder only is notified if the user clicked
    - each bidder is notified of if they won or not and the winning price
    value.
4. Winning bidder's balance is increased by 1 only if the User
clicked and decreased by the winning price regardless.


## Architecture

#### Two python files: auction.py and bidder.py


## auction.py

The auction.py file contains 2 classes: user & auction

#### 1. User
4. They are shown the ad (run show add function) for the selected User and they
either click or not based on its probability. return true or false if the clicked or not

#### 2. Auction
1. Choose a user at random: rand (user list) and return that (theta 1)
2. Auction has bidder list, (theta n):
    - for each bidder, give User id of selected random user with bid method of specific
    bidder (input user id to each bidder instance) (i think im supposed to do a multi armed bandit
    algorithm for the bidder in bid)
    - add all bids to a list then select the highest bid from the list and return that
    - return non negative bid amount (so means we oly return one value)
        - in loop pass user id to each bidder, put bid into bid list and keep track of bidder id
            - maybe do zip(bids, given bidderslist)
        - out loop assign highest bid to winning bid + bidder (keep track of bidder Id)
3. Winner is Bidder with highest bid. If there is a tie, one of
    the highest bidders is selected at random, each with equal
    probability ( import random random.choice([-40, 40]))
    - Winning price is second-highest bid meaning the max bid after
    the winner's bid is removed from the set of all bids
    - maybe create bids = set() each round (but then ties?)
    - Second_high = max(bids)
    - bid is removed, so try using .pop() to remove bid and set it as the second_high
    - Idea, try to sort set then pop(0) to get the 2nd highest value, but first check count of 
    that value in set to make sure its not a tie
5. Apply notify to each bidder (check bidder for the notify function)

In [525]:
"""The Second-Price Auction program models an ad auction and implements
a bidding algorithm using the multi-armed bandit strategy. The objective is
to finish the game with as high a balance as possible. It contains two
modules: auction_lastname.py (this module) and bidder_lastname.py

auction_lastname.py brief description + purpose:
is the main program. It imports functions from __ and to run the Second-
Price Auction program. The functions defined in this program ....

List of any classes, exception, functions, and any other objects exported by the module.
"""

import numpy as np
# import matplotlib as plt # comment out before submitting bc autograder cant see it
import seaborn as sns
import matplotlib.pyplot as plt 


class User:
    """Class to represent a user with a secret probability of clicking an ad."""

    def __init__(self):
        """Generating a probability between 0 and 1 from a uniform distribution."""
        self.__probability = np.random.uniform()

    def __str__(self):
        """User object with secret probability"""
        return "User object with secret probability: " + str(self.__probability )

    def __repr__(self):
        """User object with a secret likelihood of clicking on an ad"""
        return str(self)

    def show_ad(self):
        """Returns True to represent the user clicking on an ad or False otherwise"""
        return np.random.choice([True, False],
                    p = [self.__probability, 1-self.__probability])


class Auction:
    """Class to represent an online second-price ad auction with these rules:
        1. User is chosen with equal probability in each round
        2. Bidders place bids and the winner shows the User their ad.
        3. User may/may not click the ad and winning Bidder observes this.
        4. Winning Bidder's balance is increased by 1 only if the User
        clicked and decreased by the winning price regardless.
    """

    def __init__(self, users, bidders):
        """Initializing user, bidders, creates user id's and a dictionary
        to store balances for each bidder in the auction.
        """
        # There are num_users Users, numbered from 0 to num_users - 1.
        # The number corresponding to a user is its user_id. Create a
        # list of these user_ids to access in the auction.
        self.users = users
        self.user_id_list = np.arange(0,len(self.users))
        print(self.users)
        self.bidders = bidders 
        self.balances = {bidder: 0 for bidder in self.bidders} #score

        # Track the number of rounds we have for the auction
        self.auction_rounds = self.bidders[0].num_rounds #num total moves

    def __str__(self):
        """Return auction object with users and qualified bidders"""
        return "Auction object with" + str(self.users) + "users and " + str(self.bidders)

    def __repr__(self):
        """Return auction object with users and qualified bidders"""
        return str(self)

    def execute_round(self):
        """Executes a single auction round with the following steps:
            - random user selection
            - bids from every qualified bidder in the auction
            - selection of winning bidder based on maximum bid
            - selection of actual price (second-highest bid)
            - showing ad to user and finding out whether or not they click
            - notifying winning bidder of price and user outcome and updating balance
            - notifying losing bidders of price"""
        # Set max_bid, winning Bidder (winner), and winning_prize
        max_bid = 0
        winner = None
        winning_price = 0

        # Select a user at random
        rand_user_id = np.random.choice(self.user_id_list)
        print("user id at start:", rand_user_id)

        # For each Bidder, provide the randomly selected User ID to
        # make a bid (non-negative $ amount). If the Bidder's
        # balance goes below $ -1000, then the Bidder will be
        # disqualified from the Auction and further bidding. Add
        # all elligble bids and elligible bidders to an array.
        # Bidders don't know eachothers' bids.
        final_bids = np.array([])
        final_bidders = np.array([])
        for bidder in self.bidders:
            if self.balances[bidder] < -1000:
                print("Bidder OVER:", bidder,"BALANCE OVER:",self.balances[bidder])
                self.bidders.remove(bidder)
            else:
                bid = bidder.bid(rand_user_id)
                print("Bidder CHECK:", bidder,"bid CHECK:", bid)
                if bid >= 0:
                    final_bids = np.append(final_bids, bid)
                    final_bidders = np.append(final_bidders, bidder)
        print("final bids to use (out loop):", final_bids, "elligible bidders:", final_bidders)

        # If there are one or more Bidders with valid bids, have
        # auction. If not, skip auction and notify Bidders below
        if len(final_bids) > 0:

            # If only 1 Bidder submitted a valid bid, they win
            # and their bid is the max_bid and the winning_price.
            if len(final_bids) == 1:
                max_bid = final_bids[0]
                winner = final_bidders[0]
                final_bids = []
                winning_price = max_bid
                print("max_bid:", max_bid, "winner:", winner, "bids:", final_bids, "winning price:", winning_price)

            else:
                # Sort the bids array and identify ties.
                bids_sort, bids_sort_idx = np.sort(final_bids), np.argsort(final_bids)
                ties = bids_sort_idx[bids_sort == bids_sort[-1]]
                print("OG bids", final_bids, "sorted bids:", bids_sort, "sorted bids idx:", bids_sort_idx, "tie num:", ties)

                # In case of a tie, select one of the max Bidders at
                # random with equal probability, set the max_bid value
                # as the winning_prize, and remove the winner's max_bid
                # from the bids array.
                if len(ties) > 1:
                    max_bidder_idx = np.random.choice(ties)
                    max_bid =  final_bids[max_bidder_idx]
                    winner = final_bidders[max_bidder_idx]
                    final_bids = np.delete(final_bids, max_bidder_idx)
                    winning_price = max_bid

                # If no tie, set the highest bid as max_bid and the
                # corresponding Bidder as winner. Remove the max bid
                # from the bids array and assign the second highest
                # bid to winning_prize.
                else:
                    max_bidder_idx = bids_sort_idx[-1]
                    max_bid = bids_sort[-1]
                    winner = final_bidders[max_bidder_idx]
                    final_bids = np.delete(final_bids, max_bidder_idx)
                    winning_price = bids_sort[-2]
                print("# ties:", len(ties), "max bidder idx:", max_bidder_idx, "max bid:", max_bid, "winner:", winner, "new bids:", final_bids, "winning prize:", winning_price)

        # Show the winning bidder's ad to the selected User.
        clicked = self.users[rand_user_id].show_ad()
        print("user id:", rand_user_id, "corresponding User:", self.users[rand_user_id], "clicked?", clicked)

        # Notify each Bidder of the auction results.
        for bidder in self.bidders:
            # If the bidder is a winner, notify that they won,
            # the winning price and if the user clicked or not.
            # If user clicked, increase winner's balance by 1.
            # Regardless decrease winner's balance by the winning
            # price.
            if bidder == winner:
                bidder.notify(True, winning_price, clicked)
                if clicked:
                    self.balances[bidder] += 1
                self.balances[bidder] -= winning_price

            # If bidder is not winner, notify that they did not
            # win and the winning price.
            else:
                bidder.notify(False, winning_price)

          

## bidder.py

The bidder.py file has one class: Bidder


#### Bidder
- Each Bidder beigns with balance of $0. (If balance goes below -1000 dollars then
your Bidder will be disqualified from the Auction and further bidding)
2. (do in auction step 2) Run bid function: Bidders are all given user_id and get to make a bid (any
    non negative amount of $). Bidders don't know how much any
    other Bidder has bid
    - Maybe make each bid private?
5. (do in auction step 5) Run notify method for each bidder.
    - Winning bidder only is notified if the user clicked
    - Each bidder is notified of if they won or not and the winning price value.
    - (maybe do within notify? or in auction) Winning bidder's balance is increased by 1 only 
    if the User clicked and decreased by the winning price regardless.
    Code:
    - for each bidder (bidder list), 
        - if specific bidder is winnder, notif that they won the winning
        price and if user clicked the add:
            - if user clicked: increase balance dict of bidder by 1
        - out of ^ but still on winning bidder, decrease their balance in dict by winning price
        - else not winning bidder then notify they did not win and winning price

## Drafting & Testing Different Bidding Strategies

In [658]:
"""Bidder Strategy 2: Updates bidder attributes based on results
 from an auction round
"""

import numpy as np
import matplotlib as plt # comment out before submitting bc autograder cant see it
import seaborn as sns
import matplotlib.pyplot as plt 


class Bidder:
    """Class to represent a bidder in an online second-price ad auction"""

    def __init__(self, num_users, num_rounds):
        """Creates a bidder with inital balance of 0 and information
        on the number of Users in the game, number of rounds to be 
        played, and a round counter to strategize bidding behavior. 
        Sets epsilon to 0.01 to use in our greedy bidder strategy.
        """
        self.num_rounds = num_rounds
        self.round_counter = 0 #idk how we use
        self.num_users = num_users
        self.epsilon = 0.01

        # Track performance metrics to improve our bidding strategy.
        # Balance to ensure we stay above -$1000 cut off, the winning 
        # price per round, how many times we won/lost the auction with 
        # the given user, our rewards per user (the +1 if they 
        # clicked), and our score per user (win ratio: click/win), 
        # respectively to each metric below:
        self.__balance = 0
        self.price_history = np.array([0 for i in range(num_rounds)], dtype = np.float64)
        self.win_per_user = np.array([0 for i in range(num_users)])
        self.click_per_user = np.array([0 for i in range(num_users)]) 
        self.score_per_user = np.array([0 for i in range(num_users)], dtype = np.float64)

        # Set the first price to a random value to begin auction, set
        # a max score variable to identify the max win/ratio so far,
        # a set of all the best possible users, current rounds's user id.
        self.price_history[0] = np.round(np.random.uniform(0,1), 3)
        self.max_score = 0 
        self.best_users = set()
        self.user_id = None
        # print("first price:", self.price_history[0], "all prices:", self.price_history)

    def __str__(self):
        """Return Bidder object with balance"""
        return "Bidder object with balance $" + str(self.__balance)
    
    def __repr__(self):
       """Return Bidder object with balance"""
       return str(self)

    def bid(self, user_id):
        """Returns a non-negative bid amount in dollars
        rounded to 3 decimal places. Takes the User's ID.
        user_id: number corresponding to the user, numbered
        from 0 to num_users - 1.
        """
        self.user_id = user_id
        # Set upper limit for our bid based on previous winning 
        # prices times 2 to place higher than the last bid and 
        # not risk going too negative when they subtract the price 
        # amount from our balance. There are 3 behaviors for the 
        # given user 1. With probability epsilon, we choose a bid
        # at random (explore) within 0 and the upper limit 2. Bid 
        # either uper limit or 0 based on the user's score (greedy)
        upper_limit = np.sort(self.price_history)[-1]
        print("upper limit", upper_limit)

        if np.random.uniform() < self.epsilon:
            bid = np.random.uniform(0, upper_limit)
        else:
            # If user's score is equal to our max_score, we add it 
            # to bests_users list and bid upper limit. If it's 
            # greater than it, reset best users list since this
            # user is the new best, set our new max score to this
            # user's score, and bid upper limit. If it's neither of
            # the above, but it's in best_users, bid upper limit.
            # Else if it's lower than max score, but not the lowest
            # score, bid random between 0 and upper limit. Else if 
            # its the lowest score we dont bid.
            if self.score_per_user[user_id] == self.max_score:  
                self.best_users.add(user_id)
                bid = upper_limit
            elif self.score_per_user[user_id] > self.max_score:
                self.best_users = {user_id}
                self.max_score = self.score_per_user[user_id] 
                bid = upper_limit
            elif user_id in self.best_users:
                bid = upper_limit
            else: 
                if (self.score_per_user[user_id] != 
                    np.sort(self.score_per_user)[0]):
                    bid = np.random.uniform(0, upper_limit)
                else:
                    bid = 0
        return np.round(bid, 3)

    def notify(self, auction_winner, price, clicked = None):
        """Updates bidder attributes based on results from an
        auction round: Tracks the number of rounds so far, their
        wins/losses, and if Bidder is a winner, it updates their
        balance: +1 if user clicked, minus winning price regardless

        auction_winner: True == won, False == lost (boolean)
        price: Amount of second bid which winner pays (float)
        clicked: If Bidder won, Clicked == True/False, if 
            bidder lost, Clicked will always be None.
        """
        # If user clicked, increase winner's balance by 1. 
        # Regardless decrease winner's balance by winning price.
        if auction_winner:
            if clicked:
                self.click_per_user[self.user_id] += 1
                self.__balance += 1
            self.__balance -= np.round(price, 3)

        # Track metrics for our bidding strategy then reset user id
        self.win_per_user[self.user_id] += 1
        self.price_history = np.append(self.price_history, price)
        self.round_counter += 1
        self.score_per_user[self.user_id] = (self.click_per_user[self.user_id]
                                            /self.win_per_user[self.user_id])
        self.user_id = None

In [659]:
"""Bidder Strategy 3: Uses more detailed computation to bid
"""

import numpy as np
import matplotlib as plt # comment out before submitting bc autograder cant see it
import seaborn as sns
import matplotlib.pyplot as plt 


class MathBidder:
    """Class to represent a bidder in an online second-price ad auction"""

    def __init__(self, num_users, num_rounds):
        """Creates a bidder with inital balance of 0 and information
        on the number of Users in the game, number of rounds to be 
        played, and a round counter to strategize bidding behavior. 
        Sets epsilon to 0.01 to use in our greedy bidder strategy.
        """
        self.num_rounds = num_rounds
        self.round_counter = 0 #idk how we use
        self.num_users = num_users
        self.epsilon = 0.01

        # Track performance metrics to improve our bidding strategy.
        # Balance to ensure we stay above -$1000 cut off, the winning 
        # price per round, how many times we won/lost the auction with 
        # the given user, our rewards per user (the +1 if they 
        # clicked), and our score per user (win ratio: click/win), 
        # respectively to each metric below:
        self.__balance = 0
        self.price_history = np.array([0 for i in range(num_rounds)], dtype = np.float64)
        self.win_per_user = np.array([0 for i in range(num_users)])
        self.click_per_user = np.array([0 for i in range(num_users)]) 
        self.score_per_user = np.array([0 for i in range(num_users)], dtype = np.float64)

        # Set the first price to a random value to begin auction, set
        # a max score variable to identify the max win/ratio so far,
        # a set of all the best possible users, current rounds's user id.
        self.price_history[0] = np.round(np.random.uniform(0,1), 3)
        self.max_score = 0 
        self.best_users = set()
        self.user_id = None
        # print("first price:", self.price_history[0], "all prices:", self.price_history)

    def __str__(self):
        """Return Bidder object with balance"""
        return "MathBidder object with balance $" + str(self.__balance)
    
    def __repr__(self):
       """Return Bidder object with balance"""
       return str(self)

    def bid(self, user_id):
        """Returns a non-negative bid amount in dollars
        rounded to 3 decimal places. Takes the User's ID.
        user_id: number corresponding to the user, numbered
        from 0 to num_users - 1.
        """
        self.user_id = user_id
        # Set upper limit for our bid based on previous winning 
        # prices times 2 to place higher than the last bid and 
        # not risk going too negative when they subtract the price 
        # amount from our balance. There are 3 behaviors for the 
        # given user 1. With probability epsilon, we choose a bid
        # at random (explore) within 0 and the upper limit 2. Bid 
        # either uper limit or 0 based on the user's score (greedy)
        upper_limit = np.sort(self.price_history)[-1]
        print("upper limit", upper_limit)

        
        if np.random.uniform() < self.epsilon:
            rand_bid = np.random.uniform(0, upper_limit)
            while self.__balance - rand_bid <= -1000:
                rand_bid = np.random.uniform(0, upper_limit)
            else:
                return np.round(np.random.uniform(0, upper_limit), 3)
        else:
            # If user's score is equal to our max_score, we add it 
            # to bests_users list and bid upper limit. If it's 
            # greater than it, reset best users list since this
            # user is the new best, set our new max score to this
            # user's score, and bid upper limit. If it's neither of
            # the above, but it's in best_users, bid upper limit.
            # Else if it's lower than max score, but not the lowest
            # score, bid random between 0 and upper limit. Else if 
            # its the lowest score we dont bid.

            # First make sure the bid wont put us under $-1000 cutoff
            if self.__balance-upper_limit <= -1000:
                for i in np.arange(2,upper_limit):
                    if self.__balance-(upper_limit/i) > -1000:
                        upper_limit = np.round(upper_limit/i,3)
                        
            if self.score_per_user[user_id] == self.max_score:  
                self.best_users.add(user_id)
                return upper_limit
            elif self.score_per_user[user_id] > self.max_score:
                self.best_users = {user_id}
                self.max_score = self.score_per_user[user_id] 
                return upper_limit
            elif user_id in self.best_users:
                return upper_limit
            else: 
                if (self.score_per_user[user_id] != 
                    np.sort(self.score_per_user)[0]):
                    return np.random.uniform(0, upper_limit)
                else:
                    return 0

    def notify(self, auction_winner, price, clicked = None):
        """Updates bidder attributes based on results from an
        auction round: Tracks the number of rounds so far, their
        wins/losses, and if Bidder is a winner, it updates their
        balance: +1 if user clicked, minus winning price regardless

        auction_winner: True == won, False == lost (boolean)
        price: Amount of second bid which winner pays (float)
        clicked: If Bidder won, Clicked == True/False, if 
            bidder lost, Clicked will always be None.
        """
        # If user clicked, increase winner's balance by 1. 
        # Regardless decrease winner's balance by winning price.
        if auction_winner:
            if clicked:
                self.click_per_user[self.user_id] += 1
                self.__balance += 1
            self.__balance -= np.round(price, 3)

        # Track metrics for our bidding strategy then reset user id
        self.win_per_user[self.user_id] += 1
        self.price_history = np.append(self.price_history, price)
        self.round_counter += 1
        self.score_per_user[self.user_id] = (self.click_per_user[self.user_id]
                                            /self.win_per_user[self.user_id])
        self.user_id = None

In [669]:
users_list = [User() for i in range(5)]
bidders_list = [Bidder(len(users_list), 3), MathBidder(len(users_list), 3)]
og_bidders = [Bidder(len(users_list), 3), MathBidder(len(users_list), 3)]
auction1 = Auction(users_list, bidders_list)
for i in range(2000):
    auction1.execute_round()
print("bidder bal:" , bidders_list[0]._Bidder__balance, "math bal:", bidders_list[1]._MathBidder__balance)
print("s", bidders_list[0].score_per_user, bidders_list[0].score_per_user )
# print("w", bidders_list[0].win_per_user, bidders_list[0].win_per_user )
# print("c", bidders_list[0].click_per_user, bidders_list[0].click_per_user )
print("p", bidders_list[0].price_history, bidders_list[0].price_history )

[User object with secret probability: 0.9369935303459489, User object with secret probability: 0.425129302367917, User object with secret probability: 0.7743050686837691, User object with secret probability: 0.5847444394933251, User object with secret probability: 0.6863054242050407]
user id at start: 1
upper limit 0.548
Bidder CHECK: Bidder object with balance $0 bid CHECK: 0.548
upper limit 0.58
Bidder CHECK: MathBidder object with balance $0 bid CHECK: 0.58
final bids to use (out loop): [0.548 0.58 ] elligible bidders: [Bidder object with balance $0 MathBidder object with balance $0]
OG bids [0.548 0.58 ] sorted bids: [0.548 0.58 ] sorted bids idx: [0 1] tie num: [1]
# ties: 1 max bidder idx: 1 max bid: 0.58 winner: MathBidder object with balance $0 new bids: [0.548] winning prize: 0.548
user id: 1 corresponding User: User object with secret probability: 0.425129302367917 clicked? True
user id at start: 4
upper limit 0.548
Bidder CHECK: Bidder object with balance $0 bid CHECK: 0.548

In [522]:
"""Bidder strategy 4: best bidder so far
"""

import numpy as np
import matplotlib as plt # comment out before submitting bc autograder cant see it
import seaborn as sns
import matplotlib.pyplot as plt 


class NiceBidder:
    """Class to represent a bidder in an online second-price ad auction"""

    def __init__(self, num_users, num_rounds):
        """Creates a bidder with inital balance of 0 and information
        on the number of Users in the game, number of rounds to be 
        played, and a round counter to strategize bidding behavior. 
        Sets epsilon to 0.01 to use in our greedy bidder strategy.
        """
        self.num_rounds = num_rounds
        self.round_counter = 0 #idk how we use
        self.num_users = num_users
        self.epsilon = 0.01

        # Track performance metrics to improve our bidding strategy.
        # Balance to ensure we stay above -$1000 cut off, the winning 
        # price per round, how many times we won/lost the auction with 
        # the given user, our rewards per user (the +1 if they 
        # clicked), and our score per user (win ratio: click/win), 
        # respectively to each metric below:
        self.__balance = 0
        self.price_history = np.array([0 for i in range(num_rounds)], dtype = np.float64)
        self.win_per_user = np.array([0 for i in range(num_users)])
        self.click_per_user = np.array([0 for i in range(num_users)]) 
        self.score_per_user = np.array([0 for i in range(num_users)], dtype = np.float64)

        # Set the first price to a random value to begin auction, set
        # a max score variable to identify the max win/ratio so far,
        # a set of all the best possible users, current rounds's user id.
        self.price_history[0] = np.round(np.random.uniform(0,1), 3)
        self.max_score = 0 
        self.best_users = set()
        self.user_id = None
        # print("first price:", self.price_history[0], "all prices:", self.price_history)

    def __str__(self):
        """Return Bidder object with balance"""
        return "NiceBidder object with balance $" + str(self.__balance)
    
    def __repr__(self):
       """Return Bidder object with balance"""
       return str(self)

    def bid(self, user_id):
        """Returns a non-negative bid amount in dollars
        rounded to 3 decimal places. Takes the User's ID.
        user_id: number corresponding to the user, numbered
        from 0 to num_users - 1.
        """
        self.user_id = user_id
        # Set upper limit for our bid based on previous winning 
        # prices times 2 to place higher than the last bid and 
        # not risk going too negative when they subtract the price 
        # amount from our balance. There are 3 behaviors for the 
        # given user 1. With probability epsilon, we choose a bid
        # at random (explore) within 0 and the upper limit 2. Bid 
        # either uper limit or 0 based on the user's score (greedy)
        upper_limit = np.sort(self.price_history)[-1] * 2
        print("upper limit", upper_limit)

        if np.random.uniform() < self.epsilon:
            bid = np.random.uniform(0, upper_limit)
        else:
            # If user's score is equal to our max_score, we add it 
            # to bests_users list and bid upper limit. If it's 
            # greater than it, reset best users list since this
            # user is the new best, set our new max score to this
            # user's score, and bid upper limit. If it's neither of
            # the above, but it's in best_users, bid upper limit.
            # Else if it's lower than max score, but not the lowest
            # score, bid random between 0 and upper limit. Else if 
            # its the lowest score we want to bid low.
            if self.score_per_user[user_id] == self.max_score:  
                self.best_users.add(user_id)
                bid = upper_limit
            elif self.score_per_user[user_id] > self.max_score:
                self.best_users = {user_id}
                self.max_score = self.score_per_user[user_id] 
                bid = upper_limit
            elif user_id in self.best_users:
                bid = upper_limit
            else: 
                if self.score_per_user[user_id] != \
                    np.sort(self.score_per_user)[0]:
                    bid = np.random.uniform(0, upper_limit)
                else:
                    bid = bid = np.random.uniform(0, upper_limit/4) 
        return np.round(bid, 3)

    def notify(self, auction_winner, price, clicked = None):
        """Updates bidder attributes based on results from an
        auction round: Tracks the number of rounds so far, their
        wins/losses, and if Bidder is a winner, it updates their
        balance: +1 if user clicked, minus winning price regardless

        auction_winner: True == won, False == lost (boolean)
        price: Amount of second bid which winner pays (float)
        clicked: If Bidder won, Clicked == True/False, if 
            bidder lost, Clicked will always be None.
        """
        # If user clicked, increase winner's balance by 1. 
        # Regardless decrease winner's balance by winning price.
        if auction_winner:
            if clicked:
                self.click_per_user[self.user_id] += 1
                self.__balance += 1
            self.__balance -= np.round(price, 3)

        # Track metrics for our bidding strategy then reset user id
        self.win_per_user[self.user_id] += 1
        self.price_history = np.append(self.price_history, price)
        self.round_counter += 1
        self.score_per_user[self.user_id] = self.click_per_user[self.user_id]\
                                            /self.win_per_user[self.user_id]
        self.user_id = None

        
    def plot_history(self):
        """OPTIONAL. Creates a visual representation of how the
        auction has proceeded to assess algorithm's performance
        """
        print(self.score_per_user)
        plt.plot(self.score_per_user)
        plt.title("score per user")
        plt.show()

In [523]:
""" Bidder strategy 5: returns a bid at random
"""

import numpy as np
import matplotlib as plt # comment out before submitting bc autograder cant see it
import seaborn as sns
import matplotlib.pyplot as plt 


class RandBidder:
    """Class to represent a bidder in an online second-price ad auction"""

    def __init__(self, num_users, num_rounds):
        """Creates a bidder with inital balance of 0 and information
        on the number of Users in the game, number of rounds to be 
        played, and a round counter to strategize bidding behavior. 
        Sets epsilon to 0.01 to use in our greedy bidder strategy.
        """
        self.num_rounds = num_rounds
        self.round_counter = 0 #idk how we use
        self.num_users = num_users
        self.epsilon = 0.01

        # Track performance metrics to improve our bidding strategy.
        # Balance to ensure we stay above -$1000 cut off, the winning 
        # price per round, how many times we won/lost the auction with 
        # the given user, our rewards per user (the +1 if they 
        # clicked), and our score per user (win ratio: click/win), 
        # respectively to each metric below:
        self.__balance = 0
        self.price_history = np.array([0 for i in range(num_rounds)], dtype = np.float64)
        self.win_per_user = np.array([0 for i in range(num_users)])
        self.click_per_user = np.array([0 for i in range(num_users)]) 
        self.score_per_user = np.array([0 for i in range(num_users)], dtype = np.float64)

        # Set the first price to a random value to begin auction, set
        # a max score variable to identify the max win/ratio so far,
        # a set of all the best possible users, current rounds's user id.
        self.price_history[0] = np.round(np.random.uniform(0,1), 3)
        self.max_score = 0 
        self.best_users = set()
        self.user_id = None
        # print("first price:", self.price_history[0], "all prices:", self.price_history)

    def __str__(self):
        """Return Bidder object with balance"""
        return "Bidder object with balance $" + str(self.__balance)
    
    def __repr__(self):
       """Return Bidder object with balance"""
       return str(self)

    def bid(self, user_id):
        """Returns a non-negative bid amount in dollars
        rounded to 3 decimal places. Takes the User's ID.
        user_id: number corresponding to the user, numbered
        from 0 to num_users - 1.
        """
        self.user_id = user_id
        # Set upper limit for our bid based on previous winning 
        # prices times 2 to place higher than the last bid and 
        # not risk going too negative when they subtract the price 
        # amount from our balance. There are 3 behaviors for the 
        # given user 1. With probability epsilon, we choose a bid
        # at random (explore) within 0 and the upper limit 2. Bid 
        # either uper limit or 0 based on the user's score (greedy)
        
        return np.round(np.random.uniform(), 3)

    def notify(self, auction_winner, price, clicked = None):
        """Updates bidder attributes based on results from an
        auction round: Tracks the number of rounds so far, their
        wins/losses, and if Bidder is a winner, it updates their
        balance: +1 if user clicked, minus winning price regardless

        auction_winner: True == won, False == lost (boolean)
        price: Amount of second bid which winner pays (float)
        clicked: If Bidder won, Clicked == True/False, if 
            bidder lost, Clicked will always be None.
        """
        # If user clicked, increase winner's balance by 1. 
        # Regardless decrease winner's balance by winning price.
        if auction_winner:
            if clicked:
                self.click_per_user[self.user_id] += 1
                self.__balance += 1
            self.__balance -= np.round(price, 3)

        # Track metrics for our bidding strategy then reset user id
        self.win_per_user[self.user_id] += 1
        self.price_history = np.append(self.price_history, price)
        self.round_counter += 1
        self.score_per_user[self.user_id] = self.click_per_user[self.user_id]\
                                            /self.win_per_user[self.user_id]
        self.user_id = None

        
    def plot_history(self):
        """OPTIONAL. Creates a visual representation of how the
        auction has proceeded to assess algorithm's performance
        """
        print(self.score_per_user)
        plt.plot(self.score_per_user)
        plt.title("score per user")
        plt.show()

In [657]:
users_list = [User() for i in range(5)]
bidders_list = [Bidder(len(users_list), 3), MathBidder(len(users_list), 3)]
og_bidders = [Bidder(len(users_list), 3), MathBidder(len(users_list), 3)]
auction1 = Auction(users_list, bidders_list)
for i in range(3):
    auction1.execute_round()
print("b", bidders_list[0]._Bidder__balance, bidders_list[1]._MathBidder__balance)
print("s", bidders_list[0].score_per_user, bidders_list[0].score_per_user )
print("w", bidders_list[0].win_per_user, bidders_list[0].win_per_user )
print("c", bidders_list[0].click_per_user, bidders_list[0].click_per_user )
print("p", bidders_list[0].price_history, bidders_list[0].price_history )


[User object with secret probability: 0.06507725957973687, User object with secret probability: 0.3034932664190705, User object with secret probability: 0.8964048956210464, User object with secret probability: 0.83317295345448, User object with secret probability: 0.0222643620327847]
user id at start: 4
upper limit 0.558
Bidder CHECK: Bidder object with balance $0 bid CHECK: 0.558
upper limit 0.751
Bidder CHECK: MathBidder object with balance $0 bid CHECK: 0.751
final bids to use (out loop): [0.558 0.751] elligible bidders: [Bidder object with balance $0 MathBidder object with balance $0]
OG bids [0.558 0.751] sorted bids: [0.558 0.751] sorted bids idx: [0 1] tie num: [1]
# ties: 1 max bidder idx: 1 max bid: 0.751 winner: MathBidder object with balance $0 new bids: [0.558] winning prize: 0.558
user id: 4 corresponding User: User object with secret probability: 0.0222643620327847 clicked? False
user id at start: 0
upper limit 1.116
Bidder CHECK: Bidder object with balance $0 bid CHECK: 

In [645]:
d = {og_bidders[0]: bidder.count(0), og_bidders[1]: bidder.count(1), "none": bidder.count("none")}
d

{Bidder object with balance $0: 0,
 MathBidder object with balance $0: 0,
 'none': 0}

In [453]:
# all of these work correctly:
# print("# ties:", len(ties), "max bidder idx:", max_bidder_idx, "max bid:", max_bid, "winner:", winner, "new bids:", bids, "winning prize:", winning_price)
# print("max_bid:", max_bid, "winner:", winner, "bids:", bids, "winning price:", winning_price)
# print("OG bids", bids, "sorted bids:", bids_sort, "sorted bids idx:", bids_sort_idx, "tie num:", ties)

users_list = [User() for i in range(5)]
# bidders_list = [Bidder(len(users_list), 3) for i in range(8)]
bidders_list = [Bidder(len(users_list), 3), NiceBidder(len(users_list), 3),RandBidder(len(users_list), 3) ]
#bidders_list = [Bidder(len(users_list), 3)]
# users_list[1]._User__probability #to see probability
auction1 = Auction(users_list, bidders_list)
for rounds in range(bidders_list[0].num_rounds):
    auction1.execute_round()

first price: 0.209 all prices: [0.209 0.    0.   ]
first price: 0.576 all prices: [0.576 0.    0.   ]
first price: 0.787 all prices: [0.787 0.    0.   ]
[User object with secret probability: 0.7968599603325281, User object with secret probability: 0.4100352696957783, User object with secret probability: 0.9891736776372372, User object with secret probability: 0.8468731187436578, User object with secret probability: 0.7685236944432826]
user chosen at start: 3
upper limit 0.418
Bidder: Bidder object with balance $0 bid: 0.418
bids in loop: [0.418]
upper limit 1.152
Bidder: Bidder object with balance $0 bid: 1.152
bids in loop: [0.418 1.152]
Bidder: Bidder object with balance $0 bid: 0.26
bids in loop: [0.418 1.152 0.26 ]
final bids to use (out loop): [0.418 1.152 0.26 ] elligible bidders: [Bidder object with balance $0 Bidder object with balance $0
 Bidder object with balance $0]
OG bids [0.418 1.152 0.26 ] sorted bids: [0.26  0.418 1.152] sorted bids idx: [2 0 1] tie num: [1]
# ties: 1 